In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf

In [0]:
pip install json-lines

In [0]:
pip install sentence-transformers

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 450kB 10.0MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
     |████████████████████████████████| 870kB 44.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=05dc2a7d9519a87c23c362c4d53a970bc6b4809f61d6dc1eb945cfe3bef7d1e1
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=dde871bf72cb5f8fe1d8d258c328e3f1b891a143a9550da7d012e88df73e0271
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [0]:
#@title Config
"""
Configuration file for the project.
"""

"""
Base directory.
"""
PWD = '/content/drive/My Drive/KY, FYP/Code/'

"""
File directories.
"""
# Directory for the word embeddings
GLOVE_DIR = PWD + '/glove.6B'

# Directory for storing citation function data
DATA_DIR = PWD + '/data/data'

"""
Data files: the citation and provenance dataset.
MTL refers to the aligned dataset.
"""
DATA_FILES = {
    'func': {
        'golden_train': 'processed/golden_train.func.json',
        'golden_test': 'processed/golden_test.func.json',
    },
    'scicite': {
        'train': 'scicite/train.jsonl',
        'test': 'scicite/test.jsonl',
        'dev': 'scicite/dev.jsonl'
    },
    'acl-arc': {
        'train': 'acl-arc/train.jsonl',
        'test': 'acl-arc/test.jsonl',
        'dev': 'acl-arc/dev.jsonl'
    },
    'prov': {
        'golden_train': 'processed/golden_train.prov.json',
        'golden_test': 'processed/golden_test.prov.json',
    },
    'mtl': {
        'golden_train': 'processed/golden_train.mtl.json',
        'golden_test': 'processed/golden_test.mtl.json'
    }
}


In [0]:
#@title Data
"""
Common data operations.
"""
import json
import os
import json_lines

import numpy as np

def read_json_data(filename):
    """
    Read the given JSON file.
    """
    path = os.path.join(DATA_DIR, filename)
    with open(path, 'rb') as fp:
        content = json.load(fp)
        return content

def read_jsonl_data(filename):
    """
    Read the given JSONL file.
    """
    path = os.path.join(DATA_DIR, filename)
    content = []
    print (type(content))
    with open(path, 'rb') as fp:
        for item in json_lines.reader(fp):
            content.append(item)
        return content

"""
Custom cross validation.
"""


def compress_y(ys):
    """
    For each y in ys, if y is of the form [0 0 ... 1 ... 0], compress it to a
    single integer.
    """
    if len(ys) < 1:
        return ys

    if isinstance(ys[0], np.ndarray):
        # A hack >.<
        return map(lambda x: x.tolist().index(1), ys)
    else:
        return ys


In [0]:
datafiles = DATA_FILES['scicite']
test = read_jsonl_data(datafiles['test'])
train = read_jsonl_data(datafiles['train'])

dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

dataset = dataset_train + dataset_test

<class 'list'>
<class 'list'>


In [0]:
len(dataset)

10104

In [0]:
#@title ACL-arc (6 classes)

# import lib.logger, os, sys, random, math
import numpy as np
import os

from functools import reduce

# import config.config as config
# import data.data as data
# import data.data_func as data_func
import sklearn.metrics as metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold, train_test_split
import pandas as pd

# from tensorflow.python import debug as tf_debug

from sklearn.utils import class_weight

seed = 1020
np.random.seed(seed)
# tf.python.control_flow_ops = tf
tf.compat.v1.set_random_seed(seed)
random.seed(seed)

# import keras.backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
    GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import cosine_proximity, categorical_crossentropy

from sentence_transformers import SentenceTransformer

# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

import random
"""
Set random seed and fix bug on Dropout usage.
"""
import tensorflow as tf

import tensorflow_hub as hub

def embed_sentence(sentence):
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(sentence))
    return message_embeddings

def ilen(iterable):
    return reduce(lambda sum, element: sum + 1, iterable, 0)


def build_knn(model, output_size):
    # Flatten feature vector
    flat_dim_size = np.prod(model.output_shape[1:])
    x = Reshape(target_shape=(flat_dim_size,),
                name='features_flat')(model.output)

    # Dot product between feature vector and reference vectors
    x = Dense(units=output_size,
              activation='linear',
              use_bias=False)(x)

    classifier = Model(inputs=[model.input], outputs=x)
    return classifier

def normalize_encodings(encodings):
    ref_norms = np.linalg.norm(encodings, axis=0)
    return encodings / ref_norms



MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 100

"""
Data reading and saving from disk (so that data processing is done only once).
"""
directory = DATA_DIR
funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
               'Extends': 5}


# Function dataset start
datafiles = DATA_FILES['acl-arc']
test = read_jsonl_data(datafiles['test'])
train = read_jsonl_data((datafiles['train']))

dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

# dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
# dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
# dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
#
# dataset_func = dataset_func + dataset_func2 + dataset_func3



# embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
# print("loaded Hub Module")

tf.logging.set_verbosity(tf.logging.ERROR)

# Function dataset end
#############################################################################3

texts = list(map(lambda d: d['text'], dataset_func))

ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


print('Found %s texts.' % len(texts))

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

kf = KFold(n_splits=5)

y_pred_func_all = []
y_test_func_all = []
y_pred_prov_all = []
y_test_prov_all = []
y_pred_only_func_all = []
y_test_only_func_all = []
y_pred_only_prov_all = []
y_test_only_prov_all = []
y_pred_func = []
y_test_func = []

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

# -------------------------
texts = map(lambda d: d['text'], dataset_func)
sequences = tokenizer.texts_to_sequences(texts)
xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
ys = np.asarray(ys)

batch_num = 0
average_list = {}

x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train_unique, indices = np.unique(y_train, return_index=True)
# Proportional Reduction
# ------------------------------------

# new_x_train = []
# new_y_train = []
# arr = {}
# for index in range(len(funcs_index)):
#     print(index)
#     print("-----------------------")
#     arr[index] = []
#     for i, value in enumerate(y_train):
#         if (value == index):
#             arr[index].append(i)
#     print(len(arr[index]))
#     sample_length = len(arr[index]) / 20  # 5% of data
#     sample_length = int(sample_length)
#     for j in range(sample_length):
#         new_x_train.append(x_train[arr[index][j]])
#         new_y_train.append(y_train[arr[index][j]])
#
# new_x_train = np.asarray(new_x_train)
# new_y_train = np.asarray(new_y_train)
# x_train = new_x_train
# y_train = new_y_train

# ------------------------------------------

# Few Shot
# ------------------------------------------

# x_train_unique = [x_train[i] for i in indices]
# x_train_unique = np.asarray(x_train_unique)
# print(x_train_unique)
# y_train_unique = [y_train[i] for i in indices]
# y_train_unique = np.asarray(y_train_unique)
# x_train = x_train_unique
# y_train = y_train_unique

# ------------------------------------------

y_test = utils.to_categorical(np.asarray(y_test))
y_train = utils.to_categorical(np.asarray(y_train))

print (x_train.shape, x_test.shape, y_train.shape, y_test.shape)


NB_FILTER = 128
BATCH_SIZE = 32
count = 0
EPOCH = 15 # 20
indices = []
indices_type = []

# ---------- Only citation function ----------


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            name="embedding_layerA")

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                        name="sequence_input")
embedded_sequences = embedding_layer(sequence_input)
x = Convolution1D(filters=NB_FILTER,
                kernel_size=5,
                padding='valid',
                activation='relu',
                name='convolution_layer')(embedded_sequences)

x = GlobalMaxPooling1D(name="pooling_layer")(x)
x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
# x = Dropout(0.3)(x)
preds = Dense(len(funcs_index), activation='softmax')(x)
# preds = Dense(768, name="output_layer")(x)
# output_reshape = Reshape((1,768),name="reshape_layer")(preds)

model = Model(sequence_input, preds)

model.compile(loss=categorical_crossentropy,
            # optimizer='adam',
            optimizer='rmsprop',
            metrics=['acc'])

print(model.summary())

# import datetime
# from keras.callbacks import TensorBoard

# log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


model.load_weights('model-scicite.h5', by_name=True)

# model.fit(x_train, y_train,
#         nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

# model.save_weights('model-acl.h5')

# new_model = build_knn(model, encoded_classes.shape[1])
# print(new_model.summary())
# encoded_classes_norm = normalize_encodings(encoded_classes)
# temp_weights = new_model.get_weights()
# temp_weights[-1] = encoded_classes_norm
# new_model.set_weights(temp_weights)

y_pred_probs = model.predict(x_test)

y_pred_func = []

y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

y_test = compress_y(y_test)

y_pred_only_func_all += y_pred_func
y_test_only_func_all += y_test

        # ---------- End of citation function ----------

print('Plain_Func')
# print(average_list)
print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4))
print("Finish")

In [0]:
#@title Transferred model with Training Acl anthology (4 classes)

# New file authored 28 Jan 2018
# For citation provenance
# https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

"""
This file tests the model that is trained (on acl-arc dataset, 6 classes) that outputs a 512 dimensional vector based on USE
Using the 4 Citation taxonomy dataset, (golden_test) as the target test set
"""

"""
Perform the experiments on bootstrapped data and actual annotated data.
"""
# import lib.logger, os, sys, random, math
import numpy as np
import os

from functools import reduce

# import config.config as config
# import data.data as data
# import data.data_func as data_func
import sklearn.metrics as metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold, train_test_split
import pandas as pd

# from tensorflow.python import debug as tf_debug

from sklearn.utils import class_weight

# import keras.backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
    GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import cosine_proximity, categorical_crossentropy

from sentence_transformers import SentenceTransformer

# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

import random
"""
Set random seed and fix bug on Dropout usage.
"""
import tensorflow as tf


# import tensorflow_hub as hub

# def embed_sentence(sentence):
#   with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     message_embeddings = session.run(embed(sentence))
#     return message_embeddings

def ilen(iterable):
    return reduce(lambda sum, element: sum + 1, iterable, 0)

def build_knn(model, output_size):
  # Flatten feature vector
  flat_dim_size = np.prod(model.output_shape[1:])
  x = Reshape(target_shape=(flat_dim_size,),
              name='features_flat')(model.output)

  # Dot product between feature vector and reference vectors
  x = Dense(units=output_size,
            activation='linear',
            use_bias=False)(x)

  classifier = Model(inputs=[model.input], outputs=x)
  return classifier

def normalize_encodings(encodings):
    ref_norms = np.linalg.norm(encodings, axis=0)
    return encodings / ref_norms



seed = 1020
np.random.seed(seed)
# tf.python.control_flow_ops = tf
tf.compat.v1.set_random_seed(seed)

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50
# GLOVE_DIR = GLOVE_DIR
EMBEDDING_DIM = 100

"""
Data reading and saving from disk (so that data processing is done only once).
"""
directory = DATA_DIR
funcs_index = {'Neut': 3, 'Pos': 2, 'CoCo': 1, 'Weak': 0}


# Function dataset start
datafiles = DATA_FILES['func']
test = read_json_data(datafiles['golden_test'])
# train = data.read_jsonl_data((datafiles['train']))
train = read_json_data(datafiles['golden_train'])

dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

dataset = dataset_train + dataset_test
# dataset_func = dataset_func + dataset_func2 + dataset_func3

# random.shuffle(dataset_func)


# embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
# print("loaded Hub Module")

tf.logging.set_verbosity(tf.logging.ERROR)
# Function dataset end
#############################################################################3

texts_train = list(map(lambda d: d['context'][0], dataset_train))

texts_test = list(map(lambda d: d['context'][0], dataset_test))

texts = texts_train + texts_test

y_train = list(map(lambda d: funcs_index[d['label']], dataset_train))
y_test = list(map(lambda d: funcs_index[d['label']], dataset_test))

ys = y_train + y_test

x_train, x_test, y_train, y_test = train_test_split(texts, ys, test_size=0.20, random_state=seed)

y_train_unique, indices = np.unique(y_train, return_index=True)

print('Found %s texts.' % len(texts))

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

y_pred_func_all = []
y_test_func_all = []
y_pred_only_func_all = []
y_test_only_func_all = []
y_pred_func = []
y_test_func = []

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

# -------------------------
texts = map(lambda d: d['context'][0], dataset)
sequences = tokenizer.texts_to_sequences(texts)
xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
ys = np.asarray(ys)
xs = np.asarray(xs)

print(xs.shape, ys.shape)

sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_train = np.asarray(y_train)
x_train = np.asarray(x_train)


# Proportional Reduction
# ------------------------------------

# new_x_train = []
# new_y_train = []
# arr = {}
# for index in range(len(funcs_index)):
#     print(index)
#     print("-----------------------")
#     arr[index] = []
#     for i, value in enumerate(y_train):
#         if (value == index):
#             arr[index].append(i)
#     print(len(arr[index]))
#     sample_length = len(arr[index]) / 20  # 5% of data
#     sample_length = int(sample_length)
#     for j in range(sample_length):
#         new_x_train.append(x_train[arr[index][j]])
#         new_y_train.append(y_train[arr[index][j]])
#
# new_x_train = np.asarray(new_x_train)
# new_y_train = np.asarray(new_y_train)
# x_train = new_x_train
# y_train = new_y_train

# ------------------------------------------

# One Shot
# ------------------------------------------

# x_train_unique = [x_train[i] for i in indices]
# x_train_unique = np.asarray(x_train_unique)
# y_train_unique = [y_train[i] for i in indices]
# y_train_unique = np.asarray(y_train_unique)
# x_train = x_train_unique
# y_train = y_train_unique

# ------------------------------------------

sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)

y_test = utils.to_categorical(np.asarray(y_test))
y_train = utils.to_categorical(np.asarray(y_train))

batch_num = 0
average_list = {}


NB_FILTER = 128
BATCH_SIZE = 32
count = 0
EPOCH = 15 # 20

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            name="embedding_layerC")

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                        name="sequence_input")
embedded_sequences = embedding_layer(sequence_input)
x = Convolution1D(filters=NB_FILTER,
                kernel_size=5,
                padding='valid',
                activation='relu',
                name='convolution_layer')(embedded_sequences)

x = GlobalMaxPooling1D(name="pooling_layer")(x)
x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
# x = Dropout(0.3)(x)
preds = Dense(len(funcs_index), activation='softmax')(x)
# preds = Dense(768, name="output_layer")(x)
# output_reshape = Reshape((1,768),name="reshape_layer")(preds)

model = Model(sequence_input, preds)

model.compile(loss=categorical_crossentropy,
            # optimizer='adam',
            optimizer='rmsprop',
            metrics=['acc'])

model.load_weights('model-scicite.h5', by_name=True)

model.fit(x_train, y_train,
        nb_epoch=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2)

y_pred_probs = model.predict(x_test)
# y_test = ys

# total_diff = 0
# sample_count = 0
# for i, sample in enumerate(y_pred_probs):
#     total_diff += 1-cosine_similarity(y_pred_probs[i],ys[i])
#     sample_count += 1
# average = total_diff/sample_count
# print("The average cosine difference between the predictions and test are :")
# print(average)
# average_list[batch_num] = average
# batch_num += 1
y_pred_func = []

y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))


y_test = compress_y(y_test)

y_pred_only_func_all += y_pred_func
y_test_only_func_all += y_test

    # ---------- End of citation function ----------

print('Plain_Func')
# print(average_list)
print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4))
# print("Finish")

In [0]:
#@title Transferred model with Training SciCite (3 Classes)

"""
This file tests the model that is trained (on acl-arc dataset, 6 classes) that outputs a 512 dimensional vector based on USE
Using the 4 Citation taxonomy dataset, (golden_test) as the target test set
"""

"""
Perform the experiments on bootstrapped data and actual annotated data.
"""
# import lib.logger, os, sys, random, math
import numpy as np
import os

from functools import reduce

# import config.config as config
# import data.data as data
# import data.data_func as data_func
import sklearn.metrics as metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold, train_test_split
import pandas as pd

# from tensorflow.python import debug as tf_debug

from sklearn.utils import class_weight

# import keras.backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
    GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import cosine_proximity, categorical_crossentropy

from sentence_transformers import SentenceTransformer

import random
"""
Set random seed and fix bug on Dropout usage.
"""
import tensorflow as tf


# import tensorflow_hub as hub

# def embed_sentence(sentence):
#   with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     message_embeddings = session.run(embed(sentence))
#     return message_embeddings

def ilen(iterable):
    return reduce(lambda sum, element: sum + 1, iterable, 0)

def build_knn(model, output_size):
  # Flatten feature vector
  flat_dim_size = np.prod(model.output_shape[1:])
  x = Reshape(target_shape=(flat_dim_size,),
              name='features_flat')(model.output)

  # Dot product between feature vector and reference vectors
  x = Dense(units=output_size,
            activation='linear',
            use_bias=False)(x)

  classifier = Model(inputs=[model.input], outputs=x)
  return classifier

def normalize_encodings(encodings):
    ref_norms = np.linalg.norm(encodings, axis=0)
    return encodings / ref_norms



seed = 1020
np.random.seed(seed)
# tf.python.control_flow_ops = tf
tf.compat.v1.set_random_seed(seed)

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50
# GLOVE_DIR = GLOVE_DIR
EMBEDDING_DIM = 100

"""
Data reading and saving from disk (so that data processing is done only once).
"""
directory = DATA_DIR
funcs_index = {'background': 0, 'method': 1, 'result': 2}


# Function dataset start
datafiles = DATA_FILES['scicite']
test = read_jsonl_data(datafiles['test'])
train = read_jsonl_data(datafiles['train'])

dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

dataset = dataset_train + dataset_test

tf.logging.set_verbosity(tf.logging.ERROR)


# Function dataset end
#############################################################################3

texts_train = list(map(lambda d: d['string'], dataset_train))

texts_test = list(map(lambda d: d['string'], dataset_test))

texts = texts_train + texts_test

y_train = list(map(lambda d: funcs_index[d['label']], dataset_train))
y_test = list(map(lambda d: funcs_index[d['label']], dataset_test))

ys = y_train + y_test

x_train, x_test, y_train, y_test = train_test_split(texts, ys, test_size=0.20, random_state=42)

y_train_unique, indices = np.unique(y_train, return_index=True)

print('Found %s texts.' % len(texts))

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

y_pred_func_all = []
y_test_func_all = []
y_pred_only_func_all = []
y_test_only_func_all = []
y_pred_func = []
y_test_func = []

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_train = np.asarray(y_train)
x_train = np.asarray(x_train)


# Proportional Reduction
# ------------------------------------

# new_x_train = []
# new_y_train = []
# arr = {}
# for index in range(len(funcs_index)):
#     print(index)
#     print("-----------------------")
#     arr[index] = []
#     for i, value in enumerate(y_train):
#         if (value == index):
#             arr[index].append(i)
#     print(len(arr[index]))
#     sample_length = len(arr[index]) / 20  # 5% of data
#     sample_length = int(sample_length)
#     for j in range(sample_length):
#         new_x_train.append(x_train[arr[index][j]])
#         new_y_train.append(y_train[arr[index][j]])


# for i,element in enumerate(new_x_train):
#   new_y_train[i] = words[new_y_train[i]]

# new_x_train = np.asarray(new_x_train)
# new_y_train = np.asarray(new_y_train)
# x_train = new_x_train
# y_train = new_y_train

# ------------------------------------------

# One Shot
# ------------------------------------------

# x_train_unique = [x_train[i] for i in indices]
# x_train_unique = np.asarray(x_train_unique)
# y_train_unique = [y_train[i] for i in indices]
# y_train_unique = np.asarray(y_train_unique)
# x_train = x_train_unique
# y_train = y_train_unique

# ------------------------------------------



sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)

y_test = utils.to_categorical(np.asarray(y_test))
y_train = utils.to_categorical(np.asarray(y_train))

batch_num = 0
average_list = {}

NB_FILTER = 128
BATCH_SIZE = 32
count = 0
EPOCH = 15 # 20

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            name="embedding_layerB")

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                        name="sequence_input")
embedded_sequences = embedding_layer(sequence_input)
x = Convolution1D(filters=NB_FILTER,
                kernel_size=5,
                padding='valid',
                activation='relu',
                name='convolution_layer')(embedded_sequences)

x = GlobalMaxPooling1D(name="pooling_layer")(x)
x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
# x = Dropout(0.3)(x)
preds = Dense(len(funcs_index), activation='softmax')(x)

model = Model(sequence_input, preds)

model.compile(loss=categorical_crossentropy,
            # optimizer='adam',
            optimizer='rmsprop',
            metrics=['acc'])

# model.load_weights('model-acl.h5', by_name=True)

model.fit(x_train, y_train,
        nb_epoch=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2)

model.save_weights('model-scicite.h5')

y_pred_probs = model.predict(x_test)
# y_test = ys

# total_diff = 0
# sample_count = 0
# for i, sample in enumerate(y_pred_probs):
#     total_diff += 1-cosine_similarity(y_pred_probs[i],ys[i])
#     sample_count += 1
# average = total_diff/sample_count
# print("The average cosine difference between the predictions and test are :")
# print(average)
# average_list[batch_num] = average
# batch_num += 1
y_pred_func = []

y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

y_test = compress_y(y_test)

y_pred_only_func_all += y_pred_func
y_test_only_func_all += y_test

    # ---------- End of citation function ----------

print('Plain_Func')
# print(average_list)
print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all))
print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4))
# print("Finish")

In [0]:
#@title  SciCite Function Training: sciCite(seed,filepath,text_path)

def sciCite(seed,filepath,text_path):
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf


  # import tensorflow_hub as hub

  # def embed_sentence(sentence):
  #   with tf.Session() as session:
  #     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  #     message_embeddings = session.run(embed(sentence))
  #     return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)

  def build_knn(model, output_size):
    # Flatten feature vector
    flat_dim_size = np.prod(model.output_shape[1:])
    x = Reshape(target_shape=(flat_dim_size,),
                name='features_flat')(model.output)

    # Dot product between feature vector and reference vectors
    x = Dense(units=output_size,
              activation='linear',
              use_bias=False)(x)

    classifier = Model(inputs=[model.input], outputs=x)
    return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)

  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  # GLOVE_DIR = GLOVE_DIR
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'background': 0, 'method': 1, 'result': 2}


  # Function dataset start
  datafiles = DATA_FILES['scicite']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data(datafiles['train'])

  dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
  dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

  dataset = dataset_train + dataset_test

  tf.logging.set_verbosity(tf.logging.ERROR)


  # Function dataset end
  #############################################################################3

  texts_train = list(map(lambda d: d['string'], dataset_train))

  texts_test = list(map(lambda d: d['string'], dataset_test))

  texts = texts_train + texts_test

  y_train = list(map(lambda d: funcs_index[d['label']], dataset_train))
  y_test = list(map(lambda d: funcs_index[d['label']], dataset_test))

  ys = y_train + y_test

  x_train, x_test, y_train, y_test = train_test_split(texts, ys, test_size=0.20, random_state=seed)

  y_train_unique, indices = np.unique(y_train, return_index=True)

  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  sequences = tokenizer.texts_to_sequences(x_train)
  x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  y_train = np.asarray(y_train)
  x_train = np.asarray(x_train)


  # Proportional Reduction
  # ------------------------------------

  # new_x_train = []
  # new_y_train = []
  # arr = {}
  # for index in range(len(funcs_index)):
  #     print(index)
  #     print("-----------------------")
  #     arr[index] = []
  #     for i, value in enumerate(y_train):
  #         if (value == index):
  #             arr[index].append(i)
  #     print(len(arr[index]))
  #     sample_length = len(arr[index]) / 20  # 5% of data
  #     sample_length = int(sample_length)
  #     for j in range(sample_length):
  #         new_x_train.append(x_train[arr[index][j]])
  #         new_y_train.append(y_train[arr[index][j]])


  # for i,element in enumerate(new_x_train):
  #   new_y_train[i] = words[new_y_train[i]]

  # new_x_train = np.asarray(new_x_train)
  # new_y_train = np.asarray(new_y_train)
  # x_train = new_x_train
  # y_train = new_y_train

  # ------------------------------------------

  # One Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------



  sequences = tokenizer.texts_to_sequences(x_test)
  x_test = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  y_test = np.asarray(y_test)
  x_test = np.asarray(x_test)

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  batch_num = 0
  average_list = {}

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20

  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerB")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  # model.load_weights('model-acl.h5', by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2)

  model.save_weights(filepath)

  y_pred_probs = model.predict(x_test)
  # y_test = ys

  # total_diff = 0
  # sample_count = 0
  # for i, sample in enumerate(y_pred_probs):
  #     total_diff += 1-cosine_similarity(y_pred_probs[i],ys[i])
  #     sample_count += 1
  # average = total_diff/sample_count
  # print("The average cosine difference between the predictions and test are :")
  # print(average)
  # average_list[batch_num] = average
  # batch_num += 1
  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

      # ---------- End of citation function ----------
  path = text_path
  with open(path, "a") as text_file:
    print("Scicite Training Results Seed ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarc(seed,filepath,text_path) -- 0 shot

def aclarc(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  # new_x_train = []
  # new_y_train = []
  # arr = {}
  # for index in range(len(funcs_index)):
  #     print(index)
  #     print("-----------------------")
  #     arr[index] = []
  #     for i, value in enumerate(y_train):
  #         if (value == index):
  #             arr[index].append(i)
  #     print(len(arr[index]))
  #     sample_length = len(arr[index]) / 20  # 5% of data
  #     sample_length = int(sample_length)
  #     for j in range(sample_length):
  #         new_x_train.append(x_train[arr[index][j]])
  #         new_y_train.append(y_train[arr[index][j]])
  #
  # new_x_train = np.asarray(new_x_train)
  # new_y_train = np.asarray(new_y_train)
  # x_train = new_x_train
  # y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  print (x_train.shape, x_test.shape, y_train.shape, y_test.shape)


  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  # model.fit(x_train, y_train,
  #         nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("Aclarc testing results 0 shot ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarcF(seed,filepath,text_path) -- 5 shot

def aclarcF(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      print(index)
      print("-----------------------")
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      print(len(arr[index]))
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 5
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclarc testing result (5 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarcFull(seed,filepath,text_path) -- Full

def aclarcFull(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  # new_x_train = []
  # new_y_train = []
  # arr = {}
  # for index in range(len(funcs_index)):
  #     print(index)
  #     print("-----------------------")
  #     arr[index] = []
  #     for i, value in enumerate(y_train):
  #         if (value == index):
  #             arr[index].append(i)
  #     print(len(arr[index]))
  #     # sample_length = len(arr[index]) / 20  # 5% of data
  #     # sample_length = int(sample_length)
  #     sample_length = 5
  #     for j in range(sample_length):
  #         new_x_train.append(x_train[arr[index][j]])
  #         new_y_train.append(y_train[arr[index][j]])
  
  # new_x_train = np.asarray(new_x_train)
  # new_y_train = np.asarray(new_y_train)
  # x_train = new_x_train
  # y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclarc testing result (Full data) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarcF(seed,filepath,text_path) -- 10 shot

def aclarc10(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      print(index)
      print("-----------------------")
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      print(len(arr[index]))
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 10
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclarc testing result (10 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarcF(seed,filepath,text_path) -- 20 shot

def aclarc20(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      print(index)
      print("-----------------------")
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      print(len(arr[index]))
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 20
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclarc testing result (20 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACL-arc Testing: aclarcF(seed,filepath,text_path) -- 50 shot

def aclarc50(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
                'Extends': 5}


  # Function dataset start
  datafiles = DATA_FILES['acl-arc']
  test = read_jsonl_data(datafiles['test'])
  train = read_jsonl_data((datafiles['train']))

  dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
  dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

  # dataset_func = list(filter(lambda d: d['intent'] == 'CompareOrContrast', test + train))
  # dataset_func2 = list(filter(lambda d: d['intent'] == 'Background', test + train))
  # dataset_func3 = list(filter(lambda d: d['intent'] == 'Motivation', test + train))
  #
  # dataset_func = dataset_func + dataset_func2 + dataset_func3



  # embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
  # print("loaded Hub Module")

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['text'], dataset_func))

  ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['text'], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)
  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      print(index)
      print("-----------------------")
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      print(len(arr[index]))
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 50
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclarc testing result (50 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


========================================================


In [0]:
#@title ACLant Testing: aclarcF(seed,filepath,text_path) -- 0 shot

def aclant(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'Neut': 3, 'Pos': 2, 'CoCo': 1, 'Weak': 0}


  # Function dataset start
  datafiles = DATA_FILES['func']
  test = read_json_data(datafiles['golden_test'])
  # train = data.read_jsonl_data((datafiles['train']))
  train = read_json_data(datafiles['golden_train'])

  dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
  dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

  dataset = dataset_train + dataset_test
  dataset_func = dataset

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['context'][0], dataset_func))

  ys = list(map(lambda d: funcs_index[d['label']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['context'][0], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)

  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 5
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 15 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  # model.fit(x_train, y_train,
  #         nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclant testing result (0 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACLant Testing: aclarcF(seed,filepath,text_path) -- 5 shot

def aclantF(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'Neut': 3, 'Pos': 2, 'CoCo': 1, 'Weak': 0}


  # Function dataset start
  datafiles = DATA_FILES['func']
  test = read_json_data(datafiles['golden_test'])
  # train = data.read_jsonl_data((datafiles['train']))
  train = read_json_data(datafiles['golden_train'])

  dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
  dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

  dataset = dataset_train + dataset_test
  dataset_func = dataset

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['context'][0], dataset_func))

  ys = list(map(lambda d: funcs_index[d['label']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['context'][0], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)

  # Proportional Reduction
  # ------------------------------------

  new_x_train = []
  new_y_train = []
  arr = {}
  for index in range(len(funcs_index)):
      arr[index] = []
      for i, value in enumerate(y_train):
          if (value == index):
              arr[index].append(i)
      # sample_length = len(arr[index]) / 20  # 5% of data
      # sample_length = int(sample_length)
      sample_length = 5
      for j in range(sample_length):
          new_x_train.append(x_train[arr[index][j]])
          new_y_train.append(y_train[arr[index][j]])
  
  new_x_train = np.asarray(new_x_train)
  new_y_train = np.asarray(new_y_train)
  x_train = new_x_train
  y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 20 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclant testing result (5 Shot) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
#@title ACLant Testing: aclarcF(seed,filepath,text_path) -- Full shot

def aclantFull(seed,filepath,text_path):
  # import lib.logger, os, sys, random, math
  import numpy as np
  import os

  from functools import reduce

  # import config.config as config
  # import data.data as data
  # import data.data_func as data_func
  import sklearn.metrics as metrics
  from sklearn.metrics.pairwise import cosine_similarity
  from sklearn.model_selection import KFold, train_test_split
  import pandas as pd

  # from tensorflow.python import debug as tf_debug

  from sklearn.utils import class_weight

  # import keras.backend as K
  from tensorflow.keras import utils
  from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
      GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
  from tensorflow.keras.layers import Convolution1D, MaxPooling1D
  from tensorflow.keras.models import Model, Sequential, load_model
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  from keras.losses import cosine_proximity, categorical_crossentropy

  from sentence_transformers import SentenceTransformer

  # import matplotlib.pyplot as plt
  # from sklearn.decomposition import PCA

  import random
  """
  Set random seed and fix bug on Dropout usage.
  """
  import tensorflow as tf

  import tensorflow_hub as hub

  np.random.seed(seed)
  # tf.python.control_flow_ops = tf
  tf.compat.v1.set_random_seed(seed)
  random.seed(seed)

  def embed_sentence(sentence):
    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(sentence))
      return message_embeddings

  def ilen(iterable):
      return reduce(lambda sum, element: sum + 1, iterable, 0)


  def build_knn(model, output_size):
      # Flatten feature vector
      flat_dim_size = np.prod(model.output_shape[1:])
      x = Reshape(target_shape=(flat_dim_size,),
                  name='features_flat')(model.output)

      # Dot product between feature vector and reference vectors
      x = Dense(units=output_size,
                activation='linear',
                use_bias=False)(x)

      classifier = Model(inputs=[model.input], outputs=x)
      return classifier

  def normalize_encodings(encodings):
      ref_norms = np.linalg.norm(encodings, axis=0)
      return encodings / ref_norms



  MAX_NB_WORDS = 20000
  MAX_SEQUENCE_LENGTH = 50
  EMBEDDING_DIM = 100

  """
  Data reading and saving from disk (so that data processing is done only once).
  """
  directory = DATA_DIR
  funcs_index = {'Neut': 3, 'Pos': 2, 'CoCo': 1, 'Weak': 0}


  # Function dataset start
  datafiles = DATA_FILES['func']
  test = read_json_data(datafiles['golden_test'])
  # train = data.read_jsonl_data((datafiles['train']))
  train = read_json_data(datafiles['golden_train'])

  dataset_train = list(filter(lambda x: x['label'] != 'Error',train))
  dataset_test = list(filter(lambda x: x['label'] != 'Error', test))

  dataset = dataset_train + dataset_test
  dataset_func = dataset

  tf.logging.set_verbosity(tf.logging.ERROR)

  # Function dataset end
  #############################################################################3

  texts = list(map(lambda d: d['context'][0], dataset_func))

  ys = list(map(lambda d: funcs_index[d['label']], dataset_func))


  print('Found %s texts.' % len(texts))

  tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)

  word_index = tokenizer.word_index
  print('Found %s unique tokens.' % len(word_index))

  kf = KFold(n_splits=5)

  y_pred_func_all = []
  y_test_func_all = []
  y_pred_prov_all = []
  y_test_prov_all = []
  y_pred_only_func_all = []
  y_test_only_func_all = []
  y_pred_only_prov_all = []
  y_test_only_prov_all = []
  y_pred_func = []
  y_test_func = []

  embeddings_index = {}
  f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  # embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  # for word, i in word_index.items():
  #     embedding_vector = embeddings_index.get(word)
  #     if embedding_vector is not None:
  #         # words not found in embedding index will be all-zeros.
  #         embedding_matrix[i] = embedding_vector

  # -------------------------
  texts = map(lambda d: d['context'][0], dataset_func)
  sequences = tokenizer.texts_to_sequences(texts)
  xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  ys = np.asarray(ys)

  batch_num = 0
  average_list = {}

  x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=seed)

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train_unique, indices = np.unique(y_train, return_index=True)

  # Proportional Reduction
  # ------------------------------------

  # new_x_train = []
  # new_y_train = []
  # arr = {}
  # for index in range(len(funcs_index)):
  #     arr[index] = []
  #     for i, value in enumerate(y_train):
  #         if (value == index):
  #             arr[index].append(i)
  #     # sample_length = len(arr[index]) / 20  # 5% of data
  #     # sample_length = int(sample_length)
  #     sample_length = 5
  #     for j in range(sample_length):
  #         new_x_train.append(x_train[arr[index][j]])
  #         new_y_train.append(y_train[arr[index][j]])
  
  # new_x_train = np.asarray(new_x_train)
  # new_y_train = np.asarray(new_y_train)
  # x_train = new_x_train
  # y_train = new_y_train

  # ------------------------------------------

  # Few Shot
  # ------------------------------------------

  # x_train_unique = [x_train[i] for i in indices]
  # x_train_unique = np.asarray(x_train_unique)
  # print(x_train_unique)
  # y_train_unique = [y_train[i] for i in indices]
  # y_train_unique = np.asarray(y_train_unique)
  # x_train = x_train_unique
  # y_train = y_train_unique

  # ------------------------------------------

  y_test = utils.to_categorical(np.asarray(y_test))
  y_train = utils.to_categorical(np.asarray(y_train))

  NB_FILTER = 128
  BATCH_SIZE = 32
  count = 0
  EPOCH = 20 # 20
  indices = []
  indices_type = []

  # ---------- Only citation function ----------


  embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
      # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  embedding_layer = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              name="embedding_layerA")

  sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                          name="sequence_input")
  embedded_sequences = embedding_layer(sequence_input)
  x = Convolution1D(filters=NB_FILTER,
                  kernel_size=5,
                  padding='valid',
                  activation='relu',
                  name='convolution_layer')(embedded_sequences)

  x = GlobalMaxPooling1D(name="pooling_layer")(x)
  x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
  # x = Dropout(0.3)(x)
  preds = Dense(len(funcs_index), activation='softmax')(x)
  # preds = Dense(768, name="output_layer")(x)
  # output_reshape = Reshape((1,768),name="reshape_layer")(preds)

  model = Model(sequence_input, preds)

  model.compile(loss=categorical_crossentropy,
              # optimizer='adam',
              optimizer='rmsprop',
              metrics=['acc'])

  print(model.summary())

  # import datetime
  # from keras.callbacks import TensorBoard

  # log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


  model.load_weights(filepath, by_name=True)

  model.fit(x_train, y_train,
          nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

  # model.save_weights('model-acl.h5')

  # new_model = build_knn(model, encoded_classes.shape[1])
  # print(new_model.summary())
  # encoded_classes_norm = normalize_encodings(encoded_classes)
  # temp_weights = new_model.get_weights()
  # temp_weights[-1] = encoded_classes_norm
  # new_model.set_weights(temp_weights)

  y_pred_probs = model.predict(x_test)

  y_pred_func = []

  y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

  y_test = compress_y(y_test)

  y_pred_only_func_all += y_pred_func
  y_test_only_func_all += y_test

  path = text_path
  with open(path, "a") as text_file:
    print("aclant testing result (Full Data) ",file=text_file)
    print(seed,file=text_file)
    print(metrics.confusion_matrix(y_test_only_func_all, y_pred_only_func_all),file=text_file)
    print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4),file=text_file)
    print("=======================================================================",file=text_file)


In [0]:
seeds = [663,883,544,201,356,648,898,88,997,788]
# seeds = [1,2]
path = '/content/drive/My Drive/KY, FYP/Code/output.txt'
filepath = 'model-scicite.h5'
for seed in seeds:
  text_path = '/content/drive/My Drive/KY, FYP/Code/results/output_'
  text_path = text_path + str(seed) + ".txt"
  sciCite(seed, filepath,text_path)
  aclarc(seed, filepath,text_path)
  aclarcF(seed,filepath,text_path)
  aclarcFull(seed,filepath,text_path)

In [0]:
seeds = [663,883,544,201,356,648,898,88,997,788]
# seeds = [1,2]
path = '/content/drive/My Drive/KY, FYP/Code/output.txt'
filepath = 'model-scicite.h5'
for seed in seeds:
  text_path = '/content/drive/My Drive/KY, FYP/Code/results/output_'
  text_path = text_path + str(seed) + ".txt"
  sciCite(seed, filepath,text_path)
  aclant(seed, filepath,text_path)
  aclantF(seed,filepath,text_path)
  aclantFull(seed,filepath,text_path)

In [0]:
seeds = [663,883]
# seeds = [1,2]
path = '/content/drive/My Drive/KY, FYP/Code/output.txt'
filepath = 'model-scicite.h5'
for seed in seeds:
  text_path = '/content/drive/My Drive/KY, FYP/Code/results/output_'
  text_path = text_path + str(seed) + ".txt"
  sciCite(seed, filepath,text_path)
  aclarc(seed, filepath,text_path)
  aclarcF(seed,filepath,text_path)
  aclarc10(seed,filepath,text_path)
  aclarc20(seed,filepath,text_path)
  aclarc50(seed,filepath,text_path)

<class 'list'>
<class 'list'>
Found 10104 texts.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 35643 unique tokens.
Found 400000 word vectors.
Train on 6466 samples, validate on 1617 samples
Epoch 1/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.6993 - acc: 0.7072 - val_loss: 0.7244 - val_acc: 0.7211
Epoch 2/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.4130 - acc: 0.8388 - val_loss: 0.6561 - val_acc: 0.7916
Epoch 3/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.2843 - acc: 0.8907 - val_loss: 0.6950 - val_acc: 0.8002
Epoch 4/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.1789 - acc: 0.9357 - val_loss: 1.1858 - val_acc: 0.5986
Epoch 5/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.1049 - acc: 0.9641 - val_loss: 1.0192 - val_acc: 0.7842
Epoch 6/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.0631 - acc: 0.9816 - val_loss: 1.6010 - val_acc: 0.7434
Epoch 7/15
6466/6466 [==============================] - 7s 1ms/sample - loss: 0.04

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 400000 word vectors.
(1461, 50) (366, 50) (1461, 6) (366, 6)
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequence_input (InputLayer)  [(None, 50)]              0         
_________________________________________________________________
embedding_layerA (Embedding) (None, 50, 100)           607400    
_________________________________________________________________
convolution_layer (Conv1D)   (None, 46, 128)           64128     
_________________________________________________________________
pooling_layer (GlobalMaxPool (None, 128)               0         
_________________________________________________________________
filter_layer (Dense)         (None, 128)               16512     
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 774       
Total params: 688,814
Trainable params: 688,814
Non-train

...

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 400000 word vectors.
(1461, 50) (366, 50) (1461, 6) (366, 6)
Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequence_input (InputLayer)  [(None, 50)]              0         
_________________________________________________________________
embedding_layerA (Embedding) (None, 50, 100)           607400    
_________________________________________________________________
convolution_layer (Conv1D)   (None, 46, 128)           64128     
_________________________________________________________________
pooling_layer (GlobalMaxPool (None, 128)               0         
_________________________________________________________________
filter_layer (Dense)         (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 774       
Total params: 688,814
Trainable params: 688,814
Non-train

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'list'>
<class 'list'>
Found 1827 texts.
Found 6073 unique tokens.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [0]:
aclantF(1,'abc','a.txt')

In [0]:
#@title Combination (Aclarc + Scicite)

# Integrated Cosine Sim into the model
# https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

# import lib.logger, os, sys, random, math
import numpy as np
import os

from functools import reduce

# import config.config as config
# import data.data as data
# import data.data_func as data_func
import sklearn.metrics as metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold, train_test_split
from sklearn.utils.random import sample_without_replacement
import pandas as pd

# from tensorflow.python import debug as tf_debug

from sklearn.utils import class_weight

# import keras.backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, \
    GlobalMaxPooling1D, Embedding, Dropout, Masking, Input, Reshape
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.losses import cosine_proximity

from sentence_transformers import SentenceTransformer

# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

import random
"""
Set random seed and fix bug on Dropout usage.
"""
import tensorflow as tf

import tensorflow_hub as hub

def embed_sentence(sentence):
  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(sentence))
    return message_embeddings

def ilen(iterable):
    return reduce(lambda sum, element: sum + 1, iterable, 0)


def build_knn(model, output_size):
    # Flatten feature vector
    flat_dim_size = np.prod(model.output_shape[1:])
    x = Reshape(target_shape=(flat_dim_size,),
                name='features_flat')(model.output)

    # Dot product between feature vector and reference vectors
    x = Dense(units=output_size,
              activation='linear',
              use_bias=False)(x)

    classifier = Model(inputs=[model.input], outputs=x)
    return classifier

def normalize_encodings(encodings):
    ref_norms = np.linalg.norm(encodings, axis=0)
    return encodings / ref_norms

seed = 1020
np.random.seed(seed)
# tf.python.control_flow_ops = tf
tf.compat.v1.set_random_seed(seed)
random.seed(seed)

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 100

"""
Data reading and saving from disk (so that data processing is done only once).
"""
directory = DATA_DIR
funcs_index = {'CompareOrContrast': 0, 'Background': 1, 'Motivation': 2, 'Uses': 3, 'Future': 4,
               'Extends': 5, 'background': 6, 'method': 7, 'result': 8}


# Function dataset start
datafiles = DATA_FILES['acl-arc']
test = read_jsonl_data(datafiles['test'])
train = read_jsonl_data((datafiles['train']))

s_datafiles = DATA_FILES['scicite']
s_test = read_jsonl_data(s_datafiles['test'])
s_train = read_jsonl_data(s_datafiles['train'])

s_dataset_train = list(filter(lambda x: x['label'] != 'Error',s_train))
s_dataset_test = list(filter(lambda x: x['label'] != 'Error', s_test))

texts_train = list(map(lambda d: d['string'], s_train))

texts_test = list(map(lambda d: d['string'], s_test))

s_texts = texts_train + texts_test

y_train = list(map(lambda d: funcs_index[d['label']], s_dataset_train))
y_test = list(map(lambda d: funcs_index[d['label']], s_dataset_test))

s_ys = y_train + y_test

temp_list0 = []
temp_list1 = []
r_indices = sample_without_replacement(len(s_test + s_train),len(test + train),random_state=42)
for index in r_indices:
  temp_list0.append(s_texts[index])
  temp_list1.append(s_ys[index])
s_texts = temp_list0
s_ys = temp_list1

dataset = list(filter(lambda x: x['intent'] != 'Error', test + train))
dataset_func = list(filter(lambda d: d['intent'] != 'Error', test + train))

random.shuffle(dataset_func)


# embed = hub.Module("/content/drive/My Drive/KY, FYP/Code/USE")
# print("loaded Hub Module")

tf.logging.set_verbosity(tf.logging.ERROR)
  
s_transformer = SentenceTransformer('bert-base-nli-mean-tokens')

words = {}

# ## Raw Description
# words[0] = ["P provides relevant information for this domain."]
# words[1] = ["P illustrates need for data, goals,methods, etc."]
# words[2] = ["Uses data, methods, etc., from P"]
# words[3] = ["Extends P’s data, methods, etc. "]
# words[4] = ["Expresses similarity/differences to P."]
# words[5] = ["P is a potential avenue for future work."]

# Summarized
words[0] = ["provides relevant information"]
words[1] = ["Illustrates need"]
words[2] = ["Uses"]
words[3] = ["Extends"]
words[4] = ["similarity differences"]
words[5] = ["Potential Future"]
## Mod Description ##
words[6] = ["states background"]
words[7] = ["making use method approach"]
words[8] = ["Comparison results findings"]


# Class Title
# words[0] = ["Background"]
# words[1] = ["Motivation"]
# words[2] = ["Uses"]
# words[3] = ["Extension"]
# words[4] = ["Compare Or Contrast"]
# words[5] = ["Future"]


for i, word in enumerate(words):
    # words[i] = embed_sentence(words[i])
    words[i] = np.array(s_transformer.encode(words[i]))

# Function dataset end
#############################################################################3

texts = list(map(lambda d: d['text'], dataset_func))
texts = texts + s_texts

ys = list(map(lambda d: funcs_index[d['intent']], dataset_func))
ys = ys + s_ys

for i,element in enumerate(ys):
    ys[i] = words[ys[i]]

print('Found %s texts.' % len(texts))

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

kf = KFold(n_splits=5)

y_pred_func_all = []
y_test_func_all = []
y_pred_prov_all = []
y_test_prov_all = []
y_pred_only_func_all = []
y_test_only_func_all = []
y_pred_only_prov_all = []
y_test_only_prov_all = []
y_pred_func = []
y_test_func = []

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

# -------------------------
# texts = map(lambda d: d['text'], dataset_func)
sequences = tokenizer.texts_to_sequences(texts)
xs = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
ys = np.asarray(ys)

batch_num = 0
average_list = {}
encoded_classes = words[0]
for i in range(len(words)-1):
    encoded_classes = np.concatenate((encoded_classes,words[i+1]))
encoded_classes = encoded_classes.transpose()

x_train, x_test, y_train, y_test = train_test_split(xs, ys, test_size=0.20, random_state=42)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

NB_FILTER = 128
BATCH_SIZE = 32
count = 0
EPOCH = 15 # 20
indices = []
indices_type = []

# ---------- Only citation function ----------


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            name="embedding_layerA")

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32',
                        name="sequence_input")
embedded_sequences = embedding_layer(sequence_input)
x = Convolution1D(filters=NB_FILTER,
                kernel_size=5,
                padding='valid',
                activation='relu',
                name='convolution_layer')(embedded_sequences)

x = GlobalMaxPooling1D(name="pooling_layer")(x)
x = Dense(NB_FILTER, activation='relu',name="filter_layer")(x)
# x = Dropout(0.3)(x)
# preds = Dense(len(funcs_index), activation='softmax')(x)
preds = Dense(768, name="output_layer")(x)
output_reshape = Reshape((1,768),name="reshape_layer")(preds)

model = Model(sequence_input, output_reshape)

model.compile(loss=cosine_proximity,
            # optimizer='adam',
            optimizer='rmsprop',
            metrics=['acc'])

print(model.summary())

# import datetime
# from keras.callbacks import TensorBoard

# log_dir = "logs/few_shot/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)


# model.load_weights('model-scicite.h5', by_name=True)

model.fit(x_train, y_train,
        nb_epoch=EPOCH, batch_size=BATCH_SIZE,validation_split=0.2)

model.save_weights('model-aclarc_scicite.h5')

new_model = build_knn(model, encoded_classes.shape[1])
print(new_model.summary())
encoded_classes_norm = normalize_encodings(encoded_classes)
temp_weights = new_model.get_weights()
temp_weights[-1] = encoded_classes_norm
new_model.set_weights(temp_weights)

y_pred_probs = new_model.predict(x_test)

y_pred_func = []

y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

y_test_list = []
sim = {}

for i, sample in enumerate(y_pred_probs):
    for j in range(len(funcs_index)):
        # sim[j] = cosine_similarity(y_pred_probs[i], words[j])
        if np.array_equal(y_test[i], words[j]):
            y_test_list.append(j)
    # greatest_sim = max(sim, key=sim.get)

    # y_pred_func.append(greatest_sim)

y_test = y_test_list


# y_pred_func = list(map(lambda x: pd.Series(x).idxmax(), y_pred_probs))

# new_y_pred = [1] * len(y_pred_func)
    # Generate classificat
# y_pred_func = new_y_pred
# y_test = data.compress_y(y_test)

        #print('y_pred_func_A')
        #print(y_pred_func)

y_pred_only_func_all += y_pred_func
y_test_only_func_all += y_test

        # ---------- End of citation function ----------

print('Plain_Func')
# print(average_list)
print(metrics.classification_report(y_test_only_func_all, y_pred_only_func_all, digits=4))
print("Finish")

In [0]:
print(len(s_texts))

1827
